In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pywt
import os
from astropy.io import ascii

In [2]:
# path to GPR fit data
gprfit_path = '/local/php18ufb/backed_up_on_astro3/PTF_classification/lightcurve_GPR/GPR_fits/'
# path to GPR parameters
gprparams_path = '/local/php18ufb/backed_up_on_astro3/PTF_classification/lightcurve_GPR/GPR_params/'

# Get list of GPR fit data files
gpr_lc_list = os.listdir(gprfit_path)

# Get list of GPR parameter data files
gpr_params_list = os.listdir(gprparams_path)

In [3]:
# Method to get GPR lightcurve fit data
def get_gprlc(path, filename):
    lc_fit = ascii.read(path + filename)
    epoch_fit = lc_fit['MJD']
    mag_fit = lc_fit['mag_pred']
    mag_fit_sigma = lc_fit['mag_pred_sigma']
    
    sn_name = filename.strip('_gpr_fit.csv')
    
    return epoch_fit, mag_fit, mag_fit_sigma, sn_name

In [4]:
# Method to get GPR fit parameters
def get_gprparams(path, filename):
    params_tab = ascii.read(path + filename)
    parameters = params_tab['value']
    
    theta = parameters[0:-3]
    sn_type = parameters[-2]
    sn_filter = parameters[-1]
    
    return theta, sn_type, sn_filter

In [5]:
# Method to do 2-level wavelet decomposition, returns wavelet coefficients
def get_wavedec(x_in, y_in):
    coeffs = pywt.wavedec2([x_in, y_in], 'sym2', level=2)
    
    cA2 = coeffs[0]
    cH2, cV2, cD2 = coeffs[1][0], coeffs[1][1], coeffs[1][2]
    cH1, cV1, cD1 = coeffs[2][0], coeffs[2][1], coeffs[2][2]
    
    coeffs_vector = [cA2, cH2, cV2, cD2, cH1, cV1, cD1]
    
    return coeffs_vector

In [6]:
# Method to return total number of wavelet coefficients
def no_coeffs(coeff_vector):
    n_coeffs = 0
    
    for coeff in coeff_vector:
        x = np.shape(coeff)[0]
        y = np.shape(coeff)[1]
        
        len_coeff = x * y
        n_coeffs += len_coeff
    return n_coeffs

In [7]:
# Method to flatten vector of coefficients
def flatten_coeffs(coeff_vector):
    flattened_arr = []
    
    for coeff in coeff_vector:
        flat_coeffs = coeff.flatten()
        flattened_arr.extend(flat_coeffs)
    
    return flattened_arr

In [8]:
# Method to create a dictionary of coeff labels and values
def mk_coeff_dict(coeff_vector):
    len_cv = len(coeff_vector)
    
    coeff_labels = []
    for i in range(len_cv):
        coeff_labels.append('coeff{0}'.format(i))
    
    coeff_dict = {}
    
    for j, label in enumerate(coeff_labels):
        coeff_dict[label] = coeff_vector[j]
    
    return coeff_dict

In [9]:
sn01 = gpr_lc_list[2]

x_fit01, y_fit01, y_fit_err01, sn02_name = get_gprlc(gprfit_path, sn01)
sn01_coeffs = get_wavedec(x_fit01, y_fit01)
n_sn01_coeffs = no_coeffs(sn01_coeffs)

/usr/local/lib/python3.5/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


In [10]:
sn02 = gpr_lc_list[34]
sn02_params = gpr_params_list[34]
x_fit02, y_fit02, y_fit_err02, sn02_name = get_gprlc(gprfit_path, sn02)
sn02_theta, sn02_type, sn02_filter = get_gprparams(gprparams_path, sn02_params)

sn02_coeffs = get_wavedec(x_fit02, y_fit02)
n_sn02_coeffs = no_coeffs(sn02_coeffs)

flat_sn02_coeffs = flatten_coeffs(sn02_coeffs)

In [11]:
sn02_coeff_dict = mk_coeff_dict(flat_sn02_coeffs)

In [15]:
# Create dataframe containing the wavelet coefficients of all SNe
sn_features = []

for sn_lc in gpr_lc_list:
    x_fit, y_fit, y_fit_err, sn_name = get_gprlc(gprfit_path, sn_lc)
    #print('Wavelet decomposition for {0}'.format(sn_name))
    sn_coeffs = get_wavedec(x_fit, y_fit)
    sn_coeff_vector = flatten_coeffs(sn_coeffs)
    sn_coeff_dict = mk_coeff_dict(sn_coeff_vector)
    
    sn_features.append(sn_coeff_dict)

sn_features = pd.DataFrame(sn_features)
sn_features

,coeff0,coeff1,coeff10,coeff100,coeff101,coeff102,coeff103,coeff104,coeff105,coeff106,...,coeff90,coeff91,coeff92,coeff93,coeff94,coeff95,coeff96,coeff97,coeff98,coeff99
0,137421.125182,137423.358327,138157.218986,-48091.420045,-48120.418836,-48149.263765,-48177.819024,-48206.159527,-48234.608271,-48264.557936,...,-47804.247824,-47832.927805,-47861.581382,-47890.227869,-47918.903517,-47947.595563,-47976.295316,-48004.969938,-48033.656922,-48062.465307
1,137215.622640,137218.413256,138125.877883,-48141.449029,-48176.981273,-48212.414974,-48247.876981,-48283.536319,-48319.056870,-48356.134247,...,-47787.101253,-47822.583191,-47858.062720,-47893.539972,-47929.013857,-47964.481489,-47999.936633,-48035.366374,-48070.728050,-48106.101031
2,138220.030697,138221.276239,138628.732958,-48136.961283,-48152.834514,-48168.755344,-48184.706196,-48200.603591,-48216.435916,-48233.060176,...,-47978.106429,-47994.020448,-48009.921504,-48025.811415,-48041.696340,-48057.579227,-48073.461315,-48089.343202,-48105.223737,-48121.097651
3,137508.223978,137512.846426,138974.082512,-48632.879468,-48690.006783,-48747.131225,-48804.250282,-48861.360895,-48918.432647,-48978.250959,...,-48061.523156,-48118.680063,-48175.824002,-48232.961619,-48290.095866,-48347.228302,-48404.359730,-48461.490537,-48518.620852,-48575.750599
4,142375.088298,142375.270798,142435.280330,-49339.738852,-49342.086440,-49344.432080,-49346.777670,-49349.123230,-49351.468773,-49353.926180,...,-49316.273655,-49318.628248,-49320.976786,-49323.329897,-49325.676845,-49328.018692,-49330.388053,-49332.714786,-49335.066507,-49337.390881
5,137297.404088,137298.400682,137623.294101,-47765.719089,-47778.451843,-47791.156948,-47803.840489,-47816.509377,-47829.170834,-47842.436243,...,-47636.695640,-47649.678372,-47662.661759,-47675.637354,-47688.597099,-47701.533756,-47714.441306,-47727.315286,-47740.153041,-47752.953879
6,137629.204495,137631.390727,138350.230317,-48156.985048,-48185.084501,-48213.183206,-48241.281738,-48269.381190,-48297.454987,-48326.909664,...,-47875.999423,-47904.098233,-47932.196867,-47960.295499,-47988.394134,-48016.492752,-48044.593665,-48072.681542,-48100.788189,-48128.885510
7,137380.215623,137383.381312,138412.987217,-48286.984047,-48327.227335,-48367.470622,-48407.714070,-48447.957636,-48488.167568,-48530.430184,...,-47884.551167,-47924.794455,-47965.037743,-48005.281031,-48045.524319,-48085.767607,-48126.010895,-48166.254183,-48206.497471,-48246.740759
8,137261.705502,137264.242185,138100.364066,-48107.781042,-48140.442348,-48173.134962,-48205.823382,-48238.473514,-48271.124256,-48305.372023,...,-47781.093726,-47813.766183,-47846.438616,-47879.111047,-47911.783481,-47944.455977,-47977.128855,-48009.802344,-48042.473210,-48075.132815
9,140906.383722,140906.673040,141000.055108,-48848.162358,-48851.773814,-48855.393367,-48859.025457,-48862.672087,-48866.333080,-48870.182418,...,-48811.139178,-48814.930388,-48818.718247,-48822.491321,-48826.240203,-48829.958878,-48833.645487,-48837.302346,-48840.935253,-48844.552288
